In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import cv2
import os
from skimage.io import imread, imshow, imread_collection
from skimage.transform import resize
from skimage.feature import hog
from skimage import exposure
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
#%matplotlib inline


In [ ]:
#reading the image
img_1 = imread('/content/drive/Shareddrives/CSCE 633(Fall 2020) HW5/train_resized/img_102.jpg',as_gray=True)
imshow(img_1)
print(img_1.shape)

In [ ]:
# skimage  takes the input as height x width.
#resizing image 
resized_img_1 = resize(img_1, (128,64)) #64x128 default values of original file. 1:2 ratio important..
imshow(resized_img_1) 
print(resized_img_1.shape)

In [ ]:
#creating hog features , use default values..
fd_1, hog_image_1 = hog(resized_img_1, feature_vector=True, visualize=True)
fd_1.shape
#pixels_per_cell=3,3 --> 3218436, 8,8 -->443556

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8), sharex=True, sharey=True) 
ax1.imshow(resized_img_1, cmap=plt.cm.gray) 
ax1.set_title('Input image') 
# Rescale histogram for better display 
hog_image_rescaled_1 = exposure.rescale_intensity(hog_image_1, in_range=(0, 10)) 
ax2.imshow(hog_image_rescaled_1, cmap=plt.cm.gray) 
ax2.set_title('Histogram of Oriented Gradients')
plt.show()

In [ ]:
images = []
def load_images_from_folder(folder):
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename),cv2.IMREAD_GRAYSCALE)
        if img is not None:
            #print(img.shape) #((2823, 2823, 3))
            images.append(img)
    #return images

load_images_from_folder("/content/drive/Shared drives/CSCE 633(Fall 2020) HW5/train_resized")

In [ ]:
len(images)

In [ ]:
# skimage  takes the input as height x width.
#resizing image 
resized_images=[]
for i in images:
  r_im= resize(i, (128,64)) 
  resized_images.append(r_im)

In [ ]:
print(resized_images[1].shape)

In [ ]:
fd_list=[]
hog_images=[]
for imge in resized_images:
  fd_i, hog_image = hog(imge, feature_vector=True, visualize=True)
  fd_list.append(fd_i)
  hog_images.append(hog_image)

print(hog_image.shape)
print(fd_i) #one image as a feacture vectore array

In [ ]:
print(len(hog_images))
print(len(fd_list))
print(fd_i.shape)

In [ ]:
#can I make a new data to see what is happenening
dataframe = pd.read_csv('/content/drive/Shared drives/CSCE 633(Fall 2020) HW5/train.csv')
dataframe.head(3)
dataframe.tail(3) #249 images
dataframe = dataframe.dropna()

In [ ]:
X_train = dataframe.iloc[:, 1:-1].values #filename did not added
y_train = dataframe.iloc[:, -1].values
print(X_train.shape)


In [ ]:
np.shape(fd_list)


In [ ]:
import pandas as pd 
import csv
resultFile = open("/content/drive/MyDrive/Colab Notebooks/fd_list.csv",'w')
wr = csv.writer(resultFile)
for i in fd_list:
    wr.writerows([i])

In [ ]:
columns = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/fd_list.csv',header=None)
new_values = columns.iloc[:, 1:].values
print(new_values.shape)
columns.head(3)

In [ ]:
print(new_values.shape)
print(y_train.shape)

In [ ]:
result= np.concatenate((new_values, y_train.reshape(len(y_train),1)), axis=1)
print(result.shape)

In [ ]:
import pandas as pd 
import csv
resultwithlabel = open("/content/drive/MyDrive/Colab Notebooks/resultwithlabel.csv",'w')
wr = csv.writer(resultwithlabel)
for i in result:
    wr.writerows([i])

In [ ]:
wholedata = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/resultwithlabel.csv',header=None)
checkvalues = wholedata.iloc[:, 1:].values
checkresult = wholedata.iloc[:, -1].values
print(checkvalues.shape)
wholedata.head(3)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(checkvalues, checkresult, test_size = 0.2, random_state = 0)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
print(X_train.shape)
print(X_test.shape)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 8)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

In [ ]:
Boosting is an Ensemble learning methods that makes use of multiple learners. In boosting different algorithms work closer by learning from each other. 

In [ ]:
#importing Adaboost library and k-fold
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import KFold


In [ ]:
 kf = KFold(n_splits=5)
 #dataframe = training data
 kf.get_n_splits(X_train)


In [ ]:
print(X_train.shape)
print(y_train.shape)



In [ ]:
adaboost = AdaBoostClassifier(n_estimators=100, base_estimator= None,learning_rate=1, random_state = 1)
for train_index, test_index in kf.split(X_train):
  train_x, test_x = X_train[train_index], X_train[test_index]
  train_y, test_y = y_train[train_index], y_train[test_index]
  adaboost.fit(X_train, y_train)

In [ ]:
adaboost = AdaBoostClassifier(n_estimators=100, base_estimator= None,learning_rate=1, random_state = 1)
# base_estimator default is decision tree
# n_estimators - The maximum number of estimators at which boosting is terminated
adaboost.fit(X, y)